## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Remove STATUS and SPECIAL_CONSIDERATIONS columns since they are not beneficial (.01% and .08% variation, respectively)

# display STATUS values
print(application_df['STATUS'].value_counts())

# display SPECIAL_CONSIDERATIONS values
print(application_df['SPECIAL_CONSIDERATIONS'].value_counts())

# Drop the columns
application_df.drop(columns=['STATUS', 'SPECIAL_CONSIDERATIONS'], inplace=True)

1    34294
0        5
Name: STATUS, dtype: int64
N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64


In [5]:
# Create two categories (bins) for ASK_AMT; 5000 and >5000

# Get the maximum value for our upper bound
max_amt = application_df['ASK_AMT'].max()

# Create two bins - 0-5000 and >5000
application_df['ASK_AMT_CATEGORY'] = pd.cut(application_df['ASK_AMT'], [0, 5000, max_amt], labels=['5000', '>5000'])

# Drop original ASK_AMT column
application_df.drop(columns=['ASK_AMT'], inplace=True)

# Display the ASK_AMT_CATEGORY distribution
application_df['ASK_AMT_CATEGORY'].value_counts()

5000     25398
>5000     8901
Name: ASK_AMT_CATEGORY, dtype: int64

In [6]:
# Re-display the unique values in each column
application_df.nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
IS_SUCCESSFUL        2
ASK_AMT_CATEGORY     2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_df.value_counts('APPLICATION_TYPE')

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Create a DataFrame grouped by APPLICATION_TYPE
app_types_df = application_df.groupby(['APPLICATION_TYPE'])[['APPLICATION_TYPE']].count()

# Select the APPLICATION_TYPE records with a count less than 500
application_types_to_replace = list(app_types_df.loc[app_types_df['APPLICATION_TYPE'] < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df.value_counts('CLASSIFICATION')
classification_value_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts[classification_value_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1267        2
C1246        2
C1256        2
C3200        2
C1234        2
dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Create a DataFrame grouped by CLASSIFICATION
classification_df = application_df.groupby(['CLASSIFICATION'])[['CLASSIFICATION']].count()

# Select the CLASSIFICATION records with a count less than 1000
classifications_to_replace = list(classification_df.loc[classification_df['CLASSIFICATION'] < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_vals_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'ASK_AMT_CATEGORY'])


In [13]:
# Split our preprocessed data into our features and target arrays

# Get the feature columns; all columns except our target (IS_SUCCESSFUL)
feature_cols = application_vals_df.columns.drop('IS_SUCCESSFUL')

# Get target set
y = application_vals_df['IS_SUCCESSFUL'].values

# Get feature set
X = application_vals_df[feature_cols].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3360      
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 
Total params: 9921 (38.75 KB)
Trainable params: 9921 (38.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 7s 6ms/step - loss: 0.5706 - accuracy: 0.7200
Epoch 2/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5556 - accuracy: 0.7282
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7294
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7306
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7307
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7335
Epoch 7/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5467 - accuracy: 0.7331
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5468 - accuracy: 0.7316
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7331
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5807 - accuracy: 0.7314 - 505ms/epoch - 2ms/step
Loss: 0.5806565880775452, Accuracy: 0.7314285635948181


In [19]:
# Export our model to HDF5 file
nn.save(save_format='h5', filepath='AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
